In [1]:
tmp = open('C:/Data/PA-2/stopword_chinese.txt','r',encoding = 'utf-8')
stopword = list(tmp.read().split('\n'))
tmp.close()
tmp = open('C:/Data/PA-2/punctuation.txt','r',encoding = 'utf-8') #.read().split('\n')
punctuation = list(tmp.read().split('\n'))
tmp.close()
#print(punctuation)
for i, val in enumerate(punctuation):
    if u'\ufeff' in val:
        t = val.split('.')
        punctuation[i] = '.'

#print(punctuation)    
RemovedWords = stopword + punctuation
#print(RemovedWords)

In [2]:
%%time
import numpy as np
from collections import Counter
tmp = open('C:/Data/PA-3/TrainingData.txt', 'r',encoding = 'utf-8')
docs = list(tmp.read().split('\n'))
tmp.close()
lstTrainTag = []
lstTrainDocs = []
lstTrainTokens = []
TrainTokens = Counter()
for i, line in enumerate(docs):
    #Get topic tag
    lstTrainTag.append(line.split('\t')[1].strip()) #依照文檔讀取每一列的文章斷詞
    #Get content
    content = ""
    for tokens in line.split('\t')[2].strip().split():
        if tokens.strip().lower() not in RemovedWords:
            content += tokens.strip().lower() + " "
            TrainTokens[tokens.strip().lower()] += 1
    
    lstTrainDocs.append(content.strip()) #Remove之後的新Tokens
    
lstTrainTokens = list(TrainTokens.keys())

Wall time: 5.66 s


In [3]:
def get_TFDocToken(docs):
    mx_TF_DocToken = np.zeros([len(docs),len(lstTrainTokens)])
    for i, doc in enumerate(docs):
        for term in doc.split():
            if term in lstTrainTokens:
                mx_TF_DocToken[i,lstTrainTokens.index(term)] = doc.split().count(term)

    return mx_TF_DocToken

In [4]:
%%time
mx_DocToken = get_TFDocToken(lstTrainDocs)

Wall time: 17min 10s


In [5]:
mx_DocToken[10][lstTrainTokens.index("綠衫軍")]

6.0

In [6]:
test = mx_DocToken[0]+mx_DocToken[1]+mx_DocToken[2]+mx_DocToken[3]+mx_DocToken[4]+mx_DocToken[5]+mx_DocToken[6]+mx_DocToken[7]+mx_DocToken[8]+mx_DocToken[9]+mx_DocToken[10]+mx_DocToken[11]
#len(PoliticsVector)
test[lstTrainTokens.index("綠衫軍")]

6.0

In [7]:
%%time
numTrainDocs = len(mx_DocToken)  #總文章數.  
numWords = len(mx_DocToken[1])   #總Token數
percentSports = lstTrainTag.count("sports") / float(len(lstTrainDocs)) #Sports類文章所佔量
Politics_v_Num = np.ones(len(mx_DocToken[1])) #avoid zero probability
Sports_v_Num = np.ones(len(mx_DocToken[1])) 
PoliticsTokens = len(lstTrainTokens) #all tokens Vector size 初始值給vector size
SportsTokens = len(lstTrainTokens) #all tookens Vector size                  
for i, docs in enumerate(lstTrainDocs):
    if lstTrainTag[i] == "sports":  
        Sports_v_Num += mx_DocToken[i]  #在Spoprts類文章的所有Token的出現次數
        SportsTokens += sum(mx_DocToken[i]) #所有Sports文章的總字數
    else:  
        Politics_v_Num += mx_DocToken[i]
        PoliticsTokens += sum(mx_DocToken[i])
        
SportsVector = np.log(Sports_v_Num / SportsTokens)       # 計算 Sports 中每個 token 出現的 Probability
PoliticsVector = np.log(Politics_v_Num / PoliticsTokens)  # 計算 Politics 中每個 token 出現的 Probability  

Wall time: 15.2 s


In [8]:
SportsVector[lstTrainTokens.index("綠衫軍")]

-7.8514689508920332

In [9]:
%%time
import numpy as np
from collections import Counter
tmp = open('C:/Data/PA-3/TestData.txt', 'r',encoding = 'utf-8')
docs = list(tmp.read().split('\n'))
tmp.close()

lstTestDocs = []
for i, line in enumerate(docs):
    #Get content
    content = ""
    for tokens in line.split('\t')[1].strip().split():
        if tokens.strip().lower() not in RemovedWords:
            content += tokens.strip().lower() + " "
    
    lstTestDocs.append(content.strip()) 

Wall time: 1.96 s


In [10]:
%%time
mx_testDocToken = get_TFDocToken(lstTestDocs)

Wall time: 8min 11s


In [22]:
def MultinomialClassify(docID,testVector,SVector,PVector,pSports):
    lstCompare = [] 
    lstCompare.append(docID)
    Sports = sum(testVector * SVector) + np.log(pSports)  
    Politics = sum(testVector * PVector) + np.log(1 - pSports)
    lstCompare.append(Sports)
    lstCompare.append(Politics)
    #print(lstCompare)
    if Sports > Politics:
        lstCompare.append("Sports")
    else:
        lstCompare.append("Politics")
        
    return lstCompare

In [26]:
%%time
#print(len(mx_testDocToken))
import pandas as pd
lblColumn = ["docID","Sports_P","Politics_P","Classified_Tag"]
dfResult = pd.DataFrame(columns = lblColumn)

for i in range(len(mx_testDocToken)):
    result=MultinomialClassify(str(i+3501),mx_testDocToken[i],SportsVector,PoliticsVector,percentSports)
    #print(result)
    dfResult.loc[dfResult.shape[0]] = result
    #print(str(i+3501) + ";" + result)

Wall time: 15.9 s


In [27]:
dfResult.to_csv("HW3_Result.csv")